# Inicialización

Importar las librerías

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.impute import IterativeImputer


#semilla para poder replicar
semilla = 42
np.random.seed(semilla)

Elegimos la variable de fecha y hora 'Name', la del consumo 'kW Total' y las generales que empiezan por 'Central CO2'.

In [ ]:
carpeta = "C:\\Users\\User\\Desktop\\TFM\\DATOS"

dataframes = []

#iterar sobre los csv de la carpeta
for archivo in os.listdir(carpeta):
    ruta_completa = os.path.join(carpeta, archivo)
    #leer el CSV y añadirlo a la lista
    df_sub = pd.read_csv(ruta_completa, header=1).iloc[3:, :]
    #filtrar las columnas
    df_filt = df_sub.filter(regex='Name|kW |Central')
    dataframes.append(df_filt)

#fusionar todos los DataFrames en uno solo
df = pd.concat(dataframes, ignore_index=True) 
pd.set_option('display.max_columns', None)

df.shape

Las variables generales se denominan como 'Central CO2:variable', por lo que para simplificar, quitamos la parte anterior al ':'.

In [ ]:
#reducir nombres de las variables
df.columns = df.columns.map(lambda x: x.split(':  ')[1] if ':  ' in x else x)
df.shape
df.head(20)

,Name,kW Total,Th2 actual temp.,Th3 actual temp.,Th1 actual temp.,Th5 actual temp.,Th1 Actual state,Th2 Actual state,Th3 Actual state,Th4 Actual state,Th5 Actual state,DI 1 input,Suction temp. To-MT,Pc,Tc,Sd-MT,Ss-MT,Recalentamiento MT,Running capacity MT,Liq. inj. status MT,Trec,Prec (Recipiente),Sd-IT,Ss-IT,Recalentamiento IT,Running capacity IT,Suction temp. To-LT,Tc-BT,Sd-BT,Ss-BT,Recalentamiento BT,Running capacity LT,Cond. capacity,Pgc,Sc3,Condenser speed 1,Pgc reference,% Apertura Vhp,Pc.1,DI 2 input,Cond. temp.,Po-MT,Po-BT,Cond. ctrl.,Sgc,Cond. reference,Consigna MT,Reference MT,Suction ref. To-MT,Consigna BT,Reference LT,Suction ref. To-LT,Prec setpoint,Trec setpoint,Prec reference,Trec reference,% Apert.Vrec (Recip),Rec. low liquid leve
0,23:58:27 12/31/2023,23.4,75.28,25,14.5,-200,0,1,0,0,0,0,-6.78,57.95,21.83,64.33,7.61,14.56,56,0,0.56,33.73,74.94,22.5,22,0,-33.78,-6.78,77.5,-3,30.78,88,19,58.93,10.44,19.4,58.21,12.3,59.49,0,11.33,27.36,11.27,11.5,11.44,12.67,-8,26.51,-8,-34,11.17,-34,33.54,0.28,33.54,0.28,11.5,0
1,00:00:27 01/01/2024,23.4,78,25,16.61,-200,0,1,0,0,0,0,-8.89,57.75,21.44,62.17,7.61,16.5,51,0,-0.11,33.27,72.78,22.44,22.56,0,-34.44,-8.94,80.5,-6.28,32.33,88,35,56.78,10.56,35.5,58.21,5.9,57.18,0,13.5,26.66,10.12,13.5,13.5,12.78,-8,26.51,-8,-34,11.17,-34,33.54,0.28,33.54,0.28,7.2,0
2,00:02:27 01/01/2024,23.4,76.44,24.89,16.06,-200,0,1,0,0,0,0,-9.56,59.62,22.89,61.39,6.44,16.17,50,0,0.5,33.78,76.56,22.44,21.94,0,-37,-9.56,79.67,-7.72,29.83,57,26,59.03,10.61,25.5,58.21,8.8,59.46,0,12.11,25.03,9.88,12,12,12.61,-8,26.51,-8,-34,11.17,-34,33.54,0.28,33.54,0.28,5.3,0
3,00:04:27 01/01/2024,23.4,75.5,24.89,13.5,-200,0,1,0,0,0,0,-8.28,57.99,21.89,62.33,5.22,13.5,50,0,0.28,33.46,75.61,22.5,22.22,0,-37.78,-8.28,78.61,-5.56,29.06,57,19,58.71,10.56,18.5,58.21,10.9,59.2,0,11.72,25.9,10.39,11.78,11.78,12.56,-8,26.51,-8,-34,11.17,-34,33.54,0.28,33.54,0.28,9.2,0
4,00:06:27 01/01/2024,23.4,72.44,24.83,15,-200,0,1,0,0,0,0,-8.61,58.38,21.89,61.83,6.39,14.94,50,0,0.06,33.33,73.22,22.44,22.44,0,-34.39,-8.72,75,-3.5,28,61,27,57.23,10.44,27,58.21,6.3,57.59,0,13,26.18,11.9,13,13,12.56,-8,26.51,-8,-34,11.17,-34,33.54,0.28,33.54,0.28,7,0
5,00:08:27 01/01/2024,19.5,71.67,24.89,15.5,-200,0,1,0,0,0,0,-8,59.31,22.72,62.11,7.61,15.56,50,0,0.5,33.78,74.78,22.5,21.94,0,-32.44,-7.89,73.89,-2.5,28.89,66,22,58.97,10.39,21.5,58.21,9.4,59.37,0,12.22,25.89,12.1,12.17,12.17,12.5,-8,26.51,-8,-34,11.17,-34,33.54,0.28,33.54,0.28,4.7,0
6,00:10:27 01/01/2024,19.5,70.67,24.83,14.61,-200,0,1,0,0,0,0,-7.44,58.07,21.89,61.61,7.22,14.67,50,0,0.39,33.56,73.22,22.5,22.17,0,-31.17,-7.44,72.56,-1.72,28.72,69,22,58.35,10.44,22.3,58.21,10,58.81,0,12.39,26.98,12.6,12.5,12.5,12.5,-8,26.51,-8,-34,11.17,-34,33.54,0.28,33.54,0.28,9.3,0
7,00:12:27 01/01/2024,19.5,71.5,24.89,16.67,-200,0,1,0,0,0,0,-8.5,58.31,21.83,60.5,8.11,16.56,50,0,0.06,33.35,72.5,22.5,22.44,0,-31.56,-8.61,73.5,-1.89,31,88,27,57.52,10.28,26.9,58.21,6.9,57.89,0,12.94,26.59,11.63,12.94,12.94,12.44,-8,26.51,-8,-34,11.17,-34,33.54,0.28,33.54,0.28,8,0
8,00:14:27 01/01/2024,19.5,71.94,24.89,17.78,-200,0,1,0,0,0,0,-9.67,59.18,22.61,58.89,8.06,17.83,32,0,0.44,33.67,76,22.5,22.06,0,-33.33,-9.72,73.89,-1.83,31.61,88,21,58.93,10.33,21.4,58.21,9.2,59.38,0,12.11,25.39,11.39,12.06,12.06,12.39,-8,26.51,-8,-34,11.17,-34,33.54,0.28,33.54,0.28,5.9,0
9,00:16:27 01/01/2024,19.5,73.44,24.89,18.33,-200,0,1,0,0,0,0,-7.39,58.84,22.11,57.72,11.06,18.22,33,0,0.06,33.38,74.78,22.5,22.39,0,-33.56,-7.33,75.61,-1.72,31.22,88,19,56.82,10.33,19,58.21,3.5,57.15,0,12,27.16,11.66,11.94,11.94,12.39,-8,26.51,-8,-34,11.17,-34,33.54,0.28,33.54,0.28,4.2,0


# Preprocesado

Hemos visto que la variable objetivo kW Total se ha medido cada 10 minutos, mientras que el resto de variables cada 2 minutos. Vamos a reducir el dataset para que todas las variables estén recogidas cada 10mins. Para elegir el valor de los 5 posibles que tienen las variables medidas cada 10 mintuos, haremos la media.

Hay que cambiar el tipo de dato object a numérico par apoder hacer agregaciones. Los valores faltantes aparecen como 'Missing' en lugar de NaN, por lo que habrá que reemplazarlos por _np.nan_ para poder convertirlos en numérico.

In [ ]:
#reemplazar por np.nan
df = df.replace('Missing', np.nan)
df = df.replace('Offln', np.nan)

In [ ]:
#cambiar a datetime
df['Name'] = pd.to_datetime(df['Name'])

#lista de las columnas a cambiar
columnas_object = df.select_dtypes(include=['object']).columns

#convertir a numéricas y detectar qué columna da problemas
for columna in columnas_object:
    try:
        df[columna] = pd.to_numeric(df[columna])
    except ValueError:
        print(f"la columna '{columna}' no se ha convertido a numérica")

Ahora agrupamos las filas de 5 en 5 para remuestrear el conjunto y tener datos cada 10 minutos. En las variables contínuas se realiza la media y en las categóricas se elige la moda.

In [ ]:
#crear un índice de agrupación cada 10 minutos (5 filas)
df['Group'] = df.index // 5 

#lista de columnas que cambian cada 2 min
vars_2min = df.drop(['Name', 'kW Total'], axis=1).columns  

vars_categoricas = ['Th1 Actual state','Th2 Actual state','Th3 Actual state','Th4 Actual state','Th5 Actual state',
                    'DI 1 input','DI 2 input','Liq. inj. status MT','Rec. low liquid leve']
vars_num = list(set(vars_2min) - set(vars_categoricas))

#calcular la media o moda solo para las variables que cambian cada 2 minutos; para el resto tomar el último valor.
df_remuestreo = df.groupby('Group', as_index=False).agg({
    'Name': 'last',
    'kW Total' : 'mean', #también hacemos la media para evitar obtener valores faltantes
    **{var: 'mean' for var in vars_num},
    **{var: lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan for var in vars_categoricas} #para evitar que la moda sea un valor faltante
    })

df = df_remuestreo.drop(['Group'], axis=1)
df.head(3)

,Name,kW Total,Th2 actual temp.,Cond. ctrl.,Recalentamiento MT,Sc3,Suction ref. To-MT,Prec setpoint,Ss-MT,Pgc,Recalentamiento BT,Th1 actual temp.,Th5 actual temp.,Pc,Consigna MT,Reference MT,Suction ref. To-LT,Recalentamiento IT,Tc-BT,Trec,Running capacity MT,Pgc reference,Sd-BT,Reference LT,Sd-MT,Prec (Recipiente),Po-MT,Prec reference,Ss-IT,Suction temp. To-LT,Pc.1,Consigna BT,Ss-BT,% Apertura Vhp,Running capacity LT,Cond. temp.,Sd-IT,Po-BT,Cond. reference,Running capacity IT,Sgc,Trec setpoint,Trec reference,Th3 actual temp.,Suction temp. To-MT,Tc,Cond. capacity,% Apert.Vrec (Recip),Condenser speed 1,Th1 Actual state,Th2 Actual state,Th3 Actual state,Th4 Actual state,Th5 Actual state,DI 1 input,DI 2 input,Liq. inj. status MT,Rec. low liquid leve
0,2024-01-01 00:06:27,23.4,75.532,12.356,15.134,10.522,-8.0,33.54,6.654,58.136,30.000,15.134,-200.0,58.338,-8.0,26.51,-34.0,22.232,-8.456,0.258,51.4,58.21,78.256,11.17,62.410,33.514,26.226,33.54,22.464,-35.478,58.584,-34.0,-5.212,8.84,70.2,12.332,74.622,10.712,12.636,0.0,12.344,0.28,0.28,24.922,-8.424,21.988,25.2,8.04,25.18,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-01-01 00:16:27,19.5,71.844,12.322,16.568,10.354,-8.0,33.54,8.412,58.118,30.288,16.578,-200.0,58.742,-8.0,26.51,-34.0,22.200,-8.198,0.290,43.0,58.21,73.890,11.17,60.166,33.548,26.402,33.54,22.500,-32.412,58.520,-34.0,-1.932,7.80,79.8,12.332,74.256,11.876,12.444,0.0,12.322,0.28,0.28,24.878,-8.200,22.232,22.2,6.42,22.22,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-01-01 00:26:27,25.7,76.888,12.912,17.424,10.266,-8.0,33.54,9.900,58.126,31.156,17.434,-200.0,59.020,-8.0,26.51,-34.0,22.368,-7.512,0.154,43.2,58.21,79.112,11.17,66.522,33.478,26.680,33.54,22.510,-33.980,58.544,-34.0,-3.790,7.06,89.8,12.900,74.476,10.910,12.344,0.0,12.922,0.28,0.28,24.842,-7.524,22.264,29.0,6.60,28.84,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Guardamos el conjunto transformado.

In [ ]:
#df.to_csv("C:\\Users\\User\\Desktop\\TFM\\datos.csv", index=False)
# df = pd.read_csv("C:\\Users\\User\\Desktop\\TFM\\datos.csv")
# df['Name'] = pd.to_datetime(df['Name'])
# vars_categoricas = ['Th1 Actual state','Th2 Actual state','Th3 Actual state','Th4 Actual state','Th5 Actual state',
#                     'DI 1 input','DI 2 input','Liq. inj. status MT','Rec. low liquid leve']

Separamos conjunto en train y test de tal forma que los últimos meses pertenecen a test.

In [ ]:
num_test = int(len(df)*0.8)
df_test = df.iloc[num_test:, :].reset_index(drop=True)
df_train = df.iloc[:num_test,:].reset_index(drop=True)
print('train shape:', df_train.shape, '   test shape:',df_test.shape)
print('train: de', df_train['Name'][0], 'a', df_train['Name'][len(df_train)-1],
      '\ntest: de', df_test['Name'][0], 'a', df_test['Name'][len(df_test)-1])

train shape: (41825, 58)    test shape: (10457, 58)
train: de 2024-01-01 00:06:27 a 2024-10-19 20:48:41 
test: de 2024-10-19 20:58:41 a 2024-12-31 23:58:35


Vamos a ver en TRAIN cuantas filas hay con valores faltantes y cuantos tiene cada variable.

In [ ]:
#contar nan por columna
filas_nan = df_train.isnull().any(axis=1).sum()
porc_nan = round(filas_nan/len(df_train)*100,1)
pd.set_option('display.max_rows', 80)
print(f"Total de filas con NaNs: {filas_nan} ({porc_nan}%) \n")

nulos_por_columna = df_train.isnull().sum()
porcentaje_nulos = round(nulos_por_columna/len(df_train)*100,1)
resumen_nulos = pd.DataFrame({'Count_Nulos': nulos_por_columna, '%_Nulos': porcentaje_nulos})
resumen_nulos = resumen_nulos.sort_values('Count_Nulos', ascending=False)
print(resumen_nulos)

Total de filas con NaNs: 4574 (10.9%) 

                      Count_Nulos  %_Nulos
Tc                           4078      9.8
Po-BT                          97      0.2
Ss-BT                          94      0.2
Ss-IT                          94      0.2
Prec reference                 93      0.2
Trec reference                 93      0.2
Sd-IT                          93      0.2
Tc-BT                          93      0.2
Recalentamiento MT             92      0.2
Sd-BT                          92      0.2
Recalentamiento IT             91      0.2
Running capacity IT            90      0.2
Ss-MT                          90      0.2
Liq. inj. status MT            90      0.2
Trec                           90      0.2
Running capacity MT            89      0.2
Suction temp. To-LT            88      0.2
Th5 actual temp.               86      0.2
Prec (Recipiente)              86      0.2
Suction ref. To-LT             85      0.2
Consigna BT                    85      0.2
Th2 actual tem

Ahora en TEST

In [ ]:
#contar nan por columna
filas_nan = df_test.isnull().any(axis=1).sum()
porc_nan = round(filas_nan/len(df_test)*100,1)
pd.set_option('display.max_rows', 80)
print(f"Total de filas con NaNs: {filas_nan} ({porc_nan}%) \n")

nulos_por_columna = df_test.isnull().sum()
porcentaje_nulos = round(nulos_por_columna/len(df_test)*100,1)
resumen_nulos = pd.DataFrame({'Count_Nulos': nulos_por_columna, '%_Nulos': porcentaje_nulos})
resumen_nulos = resumen_nulos.sort_values('Count_Nulos', ascending=False)
print(resumen_nulos)

Total de filas con NaNs: 288 (2.8%) 

                      Count_Nulos  %_Nulos
Tc                            148      1.4
Ss-IT                          20      0.2
Pgc reference                  18      0.2
Suction temp. To-LT            17      0.2
Trec reference                 17      0.2
Running capacity IT            17      0.2
Sd-IT                          17      0.2
Running capacity LT            17      0.2
Po-MT                          17      0.2
Prec (Recipiente)              17      0.2
Prec reference                 17      0.2
Suction ref. To-LT             16      0.2
Consigna BT                    16      0.2
Pc                             15      0.1
Suction temp. To-MT            15      0.1
DI 2 input                     15      0.1
Th1 actual temp.               14      0.1
% Apert.Vrec (Recip)           14      0.1
Trec setpoint                  14      0.1
Cond. ctrl.                    14      0.1
Th3 actual temp.               14      0.1
Suction ref. To-

Tc tiene muchos nulos, probablemente porque el sensor ha fallado. Habrá que decididr si es conveniente quedarnos con la columna o si es mejor eliminarla. Quizás el sensor falle en ciertas circunstancias relacionadas con el consumo, por lo que podríamos usar esta información para crear la característica binaria 'Tc Error' donde 0 significa que ha medido y 1 que no ha medido, es decir, que ha habido un error.

Casi todas las variables muestran valores nulos pero son relativamente pocos, por lo que la elección de qué técnica usar para rellenarlos no perjudicará mucho a los patrones ocultos que pueda haber en los datos.

Cabe destacar que en el conjunto de train el porcentaje de valores faltantes en Tc es mucho mayor, por lo que el error podría darse más a menudo en ciertas épocas del año.

En el caso de la variable objetivo 'kW Total', en df_train, hay 205 valores nulos, y en df_test, 30.

In [ ]:
#creamos variable binaria Tc Error
df_train['Tc Error'] = df_train['Tc'].isna().astype(int)
df_test['Tc Error'] = df_test['Tc'].isna().astype(int)
vars_categoricas.append('Tc Error')

Para imputar los valores faltantes de 'kW Total' en los conjuntos de entrenamiento y test, calculamos la media del consumo correspondiente a la misma hora y día de la semana en las semanas del mismo mes. Previamente, crearemos las variables necesarias para realizar estos cálculos

In [ ]:
#TRAIN
df_train['Month'] = df_train['Name'].dt.month                                       #número del mes
df_train['Week'] = df_train['Name'].dt.isocalendar().week                           #número de la semana en el año
df_train['WeekDay'] = df_train['Name'].dt.weekday                                   #número del día en la semana
df_train['DailyIndex'] = df_train.groupby(df_train['Name'].dt.date).cumcount()     #indice de cada dato por día

#TEST
df_test['Month'] = df_test['Name'].dt.month                                     #número del mes
df_test['Week'] = df_test['Name'].dt.isocalendar().week                         #número de la semana en el año
df_test['WeekDay'] = df_test['Name'].dt.weekday                                 #número del día en la semana
df_test['DailyIndex'] = df_test.groupby(df_test['Name'].dt.date).cumcount()    #indice de cada dato por día


Ahora vamos a crear una función que realiza la imputación. Como el consumo es diferente dependiendo de la época del año, el día de la semana y la hora, los valores faltantes se imputarán teniendo en cuenta las tres semanas anteriores. No se utilizan semanas posteriores para evitar el sesgo de datos futuros. 

In [ ]:
#función que calcula la media del consumo en semanas anteriores en el mismo momento y día de la semana.
def imputar_valor(row, df):
    if pd.isnull(row['kW Total']):
        #crear df con los requisitos necesarios para calcular valor de imputación
        posibles_valores = df[
            (df['Week'] < row['Week']) &                    #semanas anteriores
            (df['Week'] >= row['Week'] - 3) &               #tres semanas atrás
            (df['WeekDay'] == row['WeekDay']) &             #mismo día de la semana
            (df['DailyIndex'] == row['DailyIndex']) &       #mismo momento
            (~df['kW Total'].isnull())                      #excluir valores nulos
        ]
        
        #asignar mayor peso a la semana más cercana
        if not posibles_valores.empty:
            #pesos inversamente proporcionales a la diferencia de semanas
            posibles_valores['Weight'] = 1 / (row['Week'] - posibles_valores['Week'] + 1)  
            #cálculo de media ponderada
            valor_imputado = np.average(posibles_valores['kW Total'], weights=posibles_valores['Weight'])
            return valor_imputado
        
    return row['kW Total']

#aplicar la función en train y test
df_train['kW Total'] = df_train.apply(lambda row: imputar_valor(row, df_train), axis=1)
df_test['kW Total'] = df_test.apply(lambda row: imputar_valor(row, df_test), axis=1)

print("Valores faltantes después de la imputación: train", df_train['kW Total'].isnull().sum(), "y test", df_test['kW Total'].isnull().sum())


Valores faltantes después de la imputación: train 0 y test 0


Ahora imputamos los valores faltantes de las columnas categóricas.

In [ ]:
#rellenamos columnas binarias
for columna in vars_categoricas:
    #rellenamos con la moda
    moda = df_train[columna].mode()[0] 
    df_train[columna].fillna(moda, inplace=True)
    df_test[columna].fillna(moda, inplace=True)
    print(f"Valores únicos en la columna {columna}:")
    print(df_train[columna].unique())
    print(df_train[columna].value_counts(normalize=True))

Valores únicos en la columna Th1 Actual state:
[0. 1.]
0.0    0.99988
1.0    0.00012
Name: Th1 Actual state, dtype: float64
Valores únicos en la columna Th2 Actual state:
[1. 0.]
1.0    0.999976
0.0    0.000024
Name: Th2 Actual state, dtype: float64
Valores únicos en la columna Th3 Actual state:
[0.]
0.0    1.0
Name: Th3 Actual state, dtype: float64
Valores únicos en la columna Th4 Actual state:
[0.]
0.0    1.0
Name: Th4 Actual state, dtype: float64
Valores únicos en la columna Th5 Actual state:
[0.]
0.0    1.0
Name: Th5 Actual state, dtype: float64
Valores únicos en la columna DI 1 input:
[0.]
0.0    1.0
Name: DI 1 input, dtype: float64
Valores únicos en la columna DI 2 input:
[0.]
0.0    1.0
Name: DI 2 input, dtype: float64
Valores únicos en la columna Liq. inj. status MT:
[0.]
0.0    1.0
Name: Liq. inj. status MT, dtype: float64
Valores únicos en la columna Rec. low liquid leve:
[0. 1.]
0.0    0.743694
1.0    0.256306
Name: Rec. low liquid leve, dtype: float64
Valores únicos en la c

Vemos las diferentes categorías que tiene cada variable y los porcentajes de cada categoría.

Eliminamos aquellos que solo tienen una única categoría, ya que no aporta información.

In [ ]:
col_elim = ['Th3 Actual state', 'Th2 Actual state', 'Th1 Actual state', 'Th4 Actual state', 'Th5 Actual state', 'DI 1 input', 'DI 2 input', 'Liq. inj. status MT']
df_train = df_train.drop(col_elim,axis=1)
df_test = df_test.drop(col_elim,axis=1)
vars_categoricas = [var for var in vars_categoricas if var not in col_elim]

Vamos a comprobar si hay variables contínuas que también tienen un único valor. En caso afirmativo se eliminarán.

In [ ]:
col_elim = []
for col in df_train.columns:
    if df_train[col].nunique() == 1:
        col_elim.append(col)
        
df_train = df_train.drop(col_elim, axis=1)
df_test = df_test.drop(col_elim, axis=1)
print(f"Columnas con un único valor eliminadas:", col_elim)

Columnas con un único valor eliminadas: ['Suction ref. To-MT', 'Prec setpoint', 'Consigna MT', 'Reference MT', 'Suction ref. To-LT', 'Reference LT', 'Prec reference', 'Consigna BT', 'Trec setpoint', 'Trec reference']


Se ha visto que en 'Th5 actual temp.' predomina un solo valor (-200).

In [ ]:
print(df_train['Th5 actual temp.'].unique())
print('count de != -200: ',len(df_train[df_train['Th5 actual temp.']!=-200]))


[-200.         nan -163.556]
count de != -200:  88


Eliminamos la varibale porque de esos 88, 84 son nan y 2 son -163.556

In [ ]:
df_train = df_train.drop('Th5 actual temp.', axis=1)
df_test = df_test.drop('Th5 actual temp.', axis=1)

In [ ]:
df_train.shape

(41825, 44)

Ahora imputamos los valores faltantes de variables fuertemente correlacionadas con otras mediante el método de iterative imputer.

In [ ]:
#función para seleccionar para cada varibale, otras variables con las que esta muy correlacionada
def ColumnasCorrelacionadas(df_tor):
   cor_matrix = df_tor.corr()
   df_nans = df_tor.loc[:, df_train.isna().any()]
   cols_nans = df_nans.columns.tolist()
   cols_dict = {}

   #seleccionar columnas con correlación mayor a 0.7, excluyendo la propia columna
   for column in cols_nans:
      cor_value = cor_matrix[column].abs()
      cor_value = cor_value[cor_value > 0.7].drop(column)
    
      #guardar las cols correlacionadas, pero si hay más de 4 seleccionar las 4 más correlacionadas
      if len(cor_value) > 0:
        if len(cor_value) <= 4:
          cols_dict[column] = cor_value.index.tolist()
        else:
          top_correlated = cor_value.nlargest(4).index.tolist()
          cols_dict[column] = top_correlated
      else:
        cols_dict[column] = []
   return cols_dict

In [ ]:
cols_excluidas = ['Name', 'kW Total', 'Month', 'Week', 'WeekDay', 'DailyIndex']
df_cor_train = df_train.drop(cols_excluidas, axis=1)

#imputación iterativa
cols_dict_train = ColumnasCorrelacionadas(df_cor_train)
for column, corr_cols in cols_dict_train.items():
    if corr_cols:  #si hay cols correlacionadas para imputar
        #añadir la col objetivo al conjunto para la imputación
        cols_imput = [column] + corr_cols
        imputer = IterativeImputer(max_iter=75,random_state=semilla)
        df_train[column] = imputer.fit_transform(df_train[cols_imput])[:, 0]
        df_test[column] = imputer.fit_transform(df_test[cols_imput])[:, 0]

    else:
        df_train[column].interpolate(method='index', inplace=True)
        df_test[column].interpolate(method='index', inplace=True)

df_train.isnull().sum().sort_values(ascending=False)

c:\Users\User\anaconda3\envs\GrAL\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
c:\Users\User\anaconda3\envs\GrAL\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
c:\Users\User\anaconda3\envs\GrAL\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Name                    0
kW Total                0
% Apertura Vhp          0
Running capacity LT     0
Cond. temp.             0
Sd-IT                   0
Po-BT                   0
Cond. reference         0
Running capacity IT     0
Sgc                     0
Th3 actual temp.        0
Suction temp. To-MT     0
Tc                      0
Cond. capacity          0
% Apert.Vrec (Recip)    0
Condenser speed 1       0
Rec. low liquid leve    0
Tc Error                0
Month                   0
Week                    0
WeekDay                 0
Ss-BT                   0
Pc.1                    0
Suction temp. To-LT     0
Pc                      0
Th2 actual temp.        0
Cond. ctrl.             0
Recalentamiento MT      0
Sc3                     0
Ss-MT                   0
Pgc                     0
Recalentamiento BT      0
Th1 actual temp.        0
Recalentamiento IT      0
Ss-IT                   0
Tc-BT                   0
Trec                    0
Running capacity MT     0
Pgc referenc

Ahora vamos a convertir las variables categóricas a int, y las demás las mantenemos con coma flotante pero solo 2 decimales, que es como aparecen en el dataset original.

In [ ]:
#convertir a INT y a dos decimales
for columna in df_train.columns:
        if columna in vars_categoricas:
                df_train[columna] = df_train[columna].astype(int)
                df_test[columna] = df_test[columna].astype(int)
        elif df_train[columna].dtype == 'float64':
                df_train[columna] = df_train[columna].round(2) 
                df_test[columna] = df_test[columna].round(2) 

Guardamos los conjuntos

In [ ]:
df_train.to_csv("C:\\Users\\User\\Desktop\\TFM\\df_train_generales.csv", index=False)
df_test.to_csv("C:\\Users\\User\\Desktop\\TFM\\df_test_generales.csv", index=False)